In [ ]:
library(DESeq2)
library(limma)
library(rtracklayer)
library(tidyverse)
library(pheatmap)
library(cowplot)
library(ComplexHeatmap)
library(stringr)
library(msigdbr)
library(tidyr)
library(circlize)
library(RColorBrewer)

### Gene lists from literature

In [ ]:
# qstem score used in https://www.nature.com/articles/s41590-022-01171-9#MOESM4
# positive and negative association
qstem_p <- as.vector(as.matrix(read.table(file.path("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_RNA_analysis/gene_lists", "qstem_pos.txt"))))
qstem_n <- as.vector(as.matrix(read.table(file.path("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_RNA_analysis/gene_lists", "qstem_neg.txt"))))
gs_names <-  as.vector(as.matrix(read.table(file.path("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_RNA_analysis/gene_lists", "msigdbr_gs_name.txt"))))
gs_names_gsea <- as.vector(as.matrix(read.table(file.path("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_RNA_analysis/gene_lists", "msigdbr_gs_name_gsea.txt"))))

# msigdbr
# get gene lists
msig_df <- msigdbr(species = "Mus musculus", category = "C7")
msig_df <- msig_df[which(msig_df$gs_name %in% gs_names), ]
# msig_df <- msig_df[grep("UP", msig_df$gs_name ), ]

# Eff vs mem
msig_df_GR_EFFvMEM_UP <- msig_df[grep("GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_UP", msig_df$gs_name ), ] # Goldrath 
msig_df_GR_EFFvMEM_DN <- msig_df[grep("GOLDRATH_EFF_VS_MEMORY_CD8_TCELL_DN", msig_df$gs_name ), ] # Goldrath 
msig_df_SK_D8_EFFvMEM_UP <- msig_df[grep("KAECH_DAY8_EFF_VS_MEMORY_CD8_TCELL_UP", msig_df$gs_name ), ] #Kaech D8
msig_df_SK_D8_EFFvMEM_DN <- msig_df[grep("KAECH_DAY8_EFF_VS_MEMORY_CD8_TCELL_DN", msig_df$gs_name ), ] #Kaech D8
msig_df_SK_D15_EFFvMEM_UP <- msig_df[grep("KAECH_DAY15_EFF_VS_MEMORY_CD8_TCELL_UP", msig_df$gs_name ), ] #Kaech D15
msig_df_SK_D15_EFFvMEM_DN <- msig_df[grep("KAECH_DAY15_EFF_VS_MEMORY_CD8_TCELL_DN", msig_df$gs_name ), ] #Kaech D15

EffvMem_UP <- unique(c(msig_df_GR_EFFvMEM_UP$gene_symbol, msig_df_SK_D8_EFFvMEM_UP$gene_symbol, msig_df_SK_D15_EFFvMEM_UP$gene_symbol))
EffvMem_DN <- unique(c(msig_df_GR_EFFvMEM_DN$gene_symbol, msig_df_SK_D8_EFFvMEM_DN$gene_symbol, msig_df_SK_D15_EFFvMEM_DN$gene_symbol))

# stem cell mem vs eff mem
msig_df_SCMvEM_UP <- msig_df[grep("STEM_CELL_MEMORY_VS_EFFECTOR_MEMORY_CD8_TCELL_UP", msig_df$gs_name ), ]
SCMvEM_UP <- unique(c(msig_df_SCMvEM_UP$gene_symbol))
SCMvEM_UP



### Input peaks

In [ ]:
# conditions below correspond to S1, S2, ..., S11 
condition = c("Naive", "Ag", "D3", "hi_hi", "hi_hi", "low_low", "low_low", "low_low", "low_hi", "low_hi", "low_hi")

In [ ]:
countData  <- read.delim("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_ATAC_analysis/eyal_pipeline/Merged_PeakFile_annotate.txt", header=T)
countData_raw <- read.delim("/data/abadiek/2023July_GScluster_copy/scifate_abadiek/Tcf7_genomics/Tcf7_ATAC_analysis/eyal_pipeline/Merged_PeakFile_annotate.txt", header=T)
countData_raw$width <- countData_raw$End - countData_raw$Start

In [ ]:
#Visualize count data row max for count threshold filtering
rm <- apply(countData[ , c(20:30) ], 1, max)
hist(rm, breaks = 1000, xlim = c(0,500))
s <- sum(rm<20)
# filter out peaks with few counts in all samples 
countData <- countData[rm >= 10, ]

# rearrange
rownames( countData ) <- countData[,1]  # countData$gene
origcolnames <- colnames(countData)
metadata <- countData[ , c(1:19) ]
countData <- countData[ , c(20:30) ]
colnames(countData) <- apply(as.data.frame(gsub(pattern="\\.\\.", replacement="|", colnames(countData))),1, function(X){ unlist(strsplit(X, "[|]"))[2]})
countData <- round(countData, 0)

# Prepare for deseq 
# first subset countData and condition to exclude the LL samples from analysis
LL_ind <- grep("low_low", condition)
condition = condition[-LL_ind]
countData = countData[, -LL_ind]

condition
head(countData)


### DESeq

In [ ]:
ExpDesign <- data.frame(row.names=colnames(countData), condition = condition)
dds = DESeqDataSetFromMatrix(countData, colData=ExpDesign, design= ~ condition)



In [ ]:
# Run DESeq if not already saved
dds1 <- DESeq(dds)

### Save or load dds

In [ ]:
saveRDS(dds1, file = "Tcf7_ATAC_full_dds_0LL.RDS")

# dds1 <- readRDS("Tcf7_ATAC_full_dds_0LL.RDS")

dat1 = cbind(metadata, counts(dds1), round(counts(dds1, normalized=TRUE),1))


### Comparison

In [ ]:
pair1 <- c("low_hi",
           "low_hi", "hi_hi",
           "low_hi", "hi_hi",
           "low_hi", "hi_hi",
           "Naive", "Naive", "Ag")
pair2 <- c("hi_hi",
           "Naive", "Naive", 
           "Ag", "Ag",
           "D3", "D3", 
           "Ag", "D3", "D3")
condtable<- matrix(c(pair1, pair2), ncol = 2)

z= length(condtable[,1])

for (i in 1:z){
    print(i)

    label = paste(condtable[i,1], "_", condtable[i,2], sep="")
    print(label)
    res <-results(dds1, cooksCutoff= FALSE,contrast=c("condition",condtable[i,1],condtable[i,2]))
    res.m=as.data.frame(res)
    colnames(res.m) = paste (label,colnames(res.m),sep=".")
    dat1<-cbind(dat1,(res.m),apply(res,1,function(x) ifelse(abs(x[2])>1 & x[6]<0.05,1,0))) # x[2] is log2FC; x[6] is adj p value
    colnames(dat1)[ncol(dat1)] <- paste0(colnames(dat1)[ncol(dat1)-1],"_pass_filter") # FDR < 0.05; log2FC > 1
}

# write.table(dat1,"Merged_PeakFile_annotate_DESEQ2_adjpvalue.txt", sep="\t", col.names=NA)

### p value visualization

In [ ]:
hist(dat1$low_hi_hi_hi.pvalue,breaks = 100)
hist(dat1$low_hi_Naive.pvalue,breaks = 100)
hist(dat1$low_hi_Ag.pvalue,breaks = 100)
hist(dat1$low_hi_D3.pvalue,breaks = 100)
hist(dat1$hi_hi_Naive.pvalue,breaks = 100)
hist(dat1$hi_hi_Ag.pvalue,breaks = 100)
hist(dat1$hi_hi_D3.pvalue,breaks = 100)
hist(dat1$Naive_Ag.pvalue,breaks = 100)
hist(dat1$Naive_D3.pvalue,breaks = 100)
hist(dat1$Ag_D3.pvalue,breaks = 100)

In [ ]:
# Variable enh between LH and HH
dat1_LH_HH <- dat1[dat1$low_hi_hi_hi.padj_pass_filter ==1, ]
dds1_LH_HH <- dds1[dat1$low_hi_hi_hi.padj_pass_filter ==1, ]

# Aggregate variable enh between each D9 sample and each control
select <- unique(c(
which(dat1$low_hi_Naive.padj_pass_filter ==1),
which(dat1$low_hi_Ag.padj_pass_filter ==1),
which(dat1$low_hi_D3.padj_pass_filter ==1),
which(dat1$hi_hi_Naive.padj_pass_filter ==1),
which(dat1$hi_hi_Ag.padj_pass_filter ==1),
which(dat1$hi_hi_D3.padj_pass_filter ==1)))
dat1_D9_c <- dat1[select, ]
dds1_D9_c <- dds1[select, ]

### Heatmap visualization 

In [ ]:

# rlog
# rld_LH_HH <- rlog(dds1_LH_HH)
rld_D9_c <- rlog(dds1_D9_c)

# select data for heatmap
rld_hm <- rld_D9_c
dat1_hm <- dat1_D9_c
dds1_hm <- dds1_D9_c

# select most significant peaks by either p value or fold change
# get minimum of all tested p value
padj_cols <- grep("padj", colnames(dat1_hm))
padj_filt_cols <- grep("padj_pass_filter", colnames(dat1_hm))
padj_cols <- padj_cols[!(padj_cols %in% padj_filt_cols)]
dat1_hm$padj_min <- apply(dat1_hm[ , padj_cols], 1, min)
# take lowest n min adj p value
top_n <- head(order(dat1_hm$padj_min), n=500)
dat1_hm <- dat1_hm[top_n, ]
dds1_hm <- dds1_hm[top_n, ]
rld_hm <- rld_hm[top_n, ]

# heatmap matrix prep
mat <- assay(rld_hm)
rownames(mat) <- dat1_hm$Gene.Name
colnames(mat)<- condition
mat_scale <- t(scale(t(mat)))



In [ ]:
# genes to label (of interest)
g_fig <- c('Tcf7', 'Cxcr4', 'Zeb1', 'Zeb2', 'Tbx21', 'PmeI', 'Ifng', 'Tnfrsf8', 'Ccl5', 'Ccr7', 'Sell', 'Hif1a', 'Nfatc1', 'Lef1', 'Runx1', 'Cxcr5', 'Pdcd1', 'Havcr2', 'Il10', 'Ctla4', 'Smad7', 'Pou6f1', 'Il6ra')
genes_to_label_pos_1 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_1 <- rownames(mat_scale[genes_to_label_pos_1, ])  

# genes to label qstem pos
g_fig <- qstem_p
genes_to_label_pos_2 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_2 <- rownames(mat_scale[genes_to_label_pos_2, ])  

# genes to label qstem neg
g_fig <- qstem_n
genes_to_label_pos_3 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_3 <- rownames(mat_scale)[genes_to_label_pos_3]  

# genes to label Goldrath eff
g_fig <- EffvMem_UP
genes_to_label_pos_4 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_4 <- rownames(mat_scale[genes_to_label_pos_4, ])  

# genes to label Goldrath mem
g_fig <- EffvMem_DN
genes_to_label_pos_5 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_5 <- rownames(mat_scale[genes_to_label_pos_5, ])  

# genes to label Goldrath mem
g_fig <- SCMvEM_UP
genes_to_label_pos_6 <- which(rownames(mat_scale) %in% g_fig )
genes_to_label_6 <- rownames(mat_scale[genes_to_label_pos_6, ])  


# column sample name annotations
labels <- c('N', 'Mem', 'Act', 'nm', 'nm', 'nem', 'nem', 'nem')
labels <- factor(labels, levels = c('Act','N', 'Mem', 'nm','nem' ))
ann <- data.frame(labels)
colnames(ann) <- c('Sample')
colors_anno = list('Sample' = c('Act' = '#A50F15', 'N' = 'green2', 'Mem' = 'darkgreen', 'nem' = 'deepskyblue1', 'nm' = 'blue2'))
colAnn <- HeatmapAnnotation(df = ann, 
  which = 'col',
  col = colors_anno, 
  annotation_width = unit(c(1, 4), 'cm'),
  gap = unit(1, 'mm'),
  annotation_legend_param = list(Sample = list(direction = "horizontal")))


ha = rowAnnotation(
                   a4=anno_mark(at= genes_to_label_pos_4, labels = genes_to_label_4, which="row", labels_gp = gpar(col= "#A50F15",fontsize = 10)),
                   a5=anno_mark(at= genes_to_label_pos_5, labels = genes_to_label_5, which="row", labels_gp = gpar(col= "darkgreen",fontsize = 10)))
#                    a6=anno_mark(at= genes_to_label_pos_6, labels = genes_to_label_6, which="row", labels_gp = gpar(col= "blue",fontsize = 10)))

#k-means cluster the peaks
set.seed(123)
split = data.frame(cutree(hclust(dist(mat_scale)), k = 4))
split = split %>% dplyr::rename(clust = 1)
split$clust <- factor(split$clust, levels = c(1,4,2,3))


# Complex heatmap

ht_list = Heatmap(mat_scale,
    col = colorRamp2(seq(from=-2, to=2, by = (2--2)/12)[1:12], viridis::viridis(12)),
    name = "scaled_rlog", 
    show_column_names = FALSE, # false when using column annotation instead 
    show_row_names = FALSE, 
    row_names_gp = gpar(fontsize = 4), 
    cluster_columns = TRUE, 
    column_dend_reorder = c(3,5,1,8,8,10,10,10), # weights for reordering dendrogram
    cluster_rows = TRUE, 
    show_column_dend = TRUE, 
    show_row_dend = FALSE,
#     show_row_dend = TRUE,
#     row_dend_width = unit(3, "cm"),
    width = unit(6, "cm"),
    height = unit(24, "cm"),
    clustering_method_rows = "complete",
                  
    # split by kmeans
    cluster_row_slices = FALSE, # manual ordering          
    row_split = split,  
    
    # remove row cluster annotations
    row_title = NULL,    
    
    # annotation
    right_annotation = ha,
    top_annotation = colAnn,
        
    # horizontal legend
    heatmap_legend_param = list(direction = "horizontal")
    
)

ht_list = draw(ht_list, annotation_legend_side="right", heatmap_legend_side = "bottom")




In [ ]:
# Save
pdf("ATAC_output_updated/ATAC_D9_hm_clustering-complete_adj-p_n500_enh-prom_clustered.pdf", width=6, height=12)
draw(ht_list)
dev.off()

### Generate PCA 

In [ ]:
# make condition factor with levels for proper plotting order
labels <- c('N', 'Mem', 'Act', 'nm', 'nm', 'nem', 'nem', 'nem')
rld_hm$condition <-  factor(rld_hm$condition, levels = c("D3", "Naive", "Ag", "hi_hi", "low_hi"))

# colors 
cols <-  c("#A50F15", "green2", "darkgreen", "blue2", "deepskyblue1")

# PCA using top 500 DEG with padj <.05 between controls and D9
pcaData <- plotPCA(rld_hm, intgroup=c("condition"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
g <- ggplot(pcaData, aes(PC1, PC2, color=condition)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"%")) +
  ylab(paste0("PC2: ",percentVar[2],"%")) + 
  coord_fixed() + scale_color_manual("",labels=c('Act', 'N', 'Mem', 'nm', 'nem'), values = cols) +
theme_bw(base_size=20) + theme(axis.text=element_text(size=14))

g
save_plot(g, file = "ATAC_output_updated/ATAC_pca_D9_c_0LL.pdf")

### Generate correlation map as alternative to PCA

In [ ]:
# choose matrix for sample distance plot
mat_dist <- rld_hm
mat_dist$cond_label <- c('N', 'Mem', 'Act', 'nm', 'nm', 'nem', 'nem', 'nem')

sampleDists <- dist(t(assay(mat_dist)))
sampleDistMatrix <- as.matrix(sampleDists)
rownames(sampleDistMatrix) <- paste(mat_dist$cond_label, mat_dist$type)
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)

ht_corr = Heatmap(sampleDistMatrix, 
    name = "Sample Dist",
    show_column_names = FALSE, 
    show_row_names = TRUE, 
    # row_names_gp = gpar(fontsize = 2), 
    cluster_columns = TRUE, 
    column_dend_reorder = c(3,5,1,8,8,10,10,10), # weights for reordering dendrogram
    cluster_rows = TRUE, 
    row_dend_reorder = c(3,5,1,8,8,10,10,10), # weights for reordering dendrogram
    show_column_dend = TRUE, show_row_dend = TRUE,width = unit(8, "cm"),
    col = colors
)
ht_corr = draw(ht_corr)

pdf("ATAC_output_updated/ATAC_corr-matrix.pdf", width=6, height=4)
ht_corr
dev.off()

### Get genomic coordinates of DE peaks of genes of interest for visualization

In [ ]:
# TFs from sci dataset
TF_Tcell <- str_to_title(c('JUNB', 'MYC', 'TBX21', 'IRF4', 'EGR1', 'NFATC1', 'BMYC', 'MXI1', 'STAT5A', 'NFAT5', 'ELK3', 'EOMES', 'REL', 'BHLHE40', 'STAT3', 'RUNX2', 'FOXO3', 'MXD4', 'BCL11B', 'CUX1', 'GTF2I', 'FOXO1', 'FLI1', 'STAT1', 'CHD2', 'ZEB1', 'FOXN3', 'TCF7', 'LEF1', 'ELF1', 'MYB', 'IKZF1', 'TCF12'))
# additional goi
goi <- c('Tcf7', 'Cxcr4', 'Zeb1', 'Zeb2', 'Tbx21', 'PmeI', 'Ifng', 'Tnfrsf8', 'Ccl5', 'Ccr7', 'Sell', 'Hif1a', 'Nfatc1', 'Lef1', 'Runx1', 'Cxcr5', 'Pdcd1', 'Havcr2', 'Il10', 'Ctla4', 'Smad7', 'Pou6f1', 'Il6ra',
         'Tbx21', 'Il7r', 'Eomes',
         'Cx3cr1', 'Klrg1', 'Bach2', 'Gzmb', 'Prf1',
         'Klrc1', 'Klrc2', 'Klrk1') # From RNA DE
goi <- unique(c(TF_Tcell, goi))
dat1_goi <- dat1_D9_c[which(dat1_D9_c$Gene.Name %in% goi), ]

# generate combined chr start end column
dat1_goi_info <- dat1_goi %>% dplyr::select(Gene.Name, Chr, Start, End, Distance.to.TSS) %>%
  unite("chr_start", Chr,Start, sep = c("-"), remove = FALSE) %>%
  unite("loc", chr_start,End, sep = c("-"), remove = FALSE) %>% dplyr::select(!chr_start) 


# label as promoter or enhancer based on distance to TSS
dat1_goi_info$cat <- "none"
dat1_goi_info[which(abs(dat1_goi_info$Distance.to.TSS) <500, ), ]$cat <- "prom"
dat1_goi_info[which(abs(dat1_goi_info$Distance.to.TSS) >=500, ), ]$cat <- "enh"


In [ ]:
head(dat1_goi_info)

In [ ]:
write_csv(dat1_goi_info, file = "ATAC_DE_peak_goi_info.csv")